<a href="https://colab.research.google.com/github/kmongsil1105/colab_ipynb/blob/main/AI_PyTorch(%ED%86%A0%EC%B9%98%ED%85%8D%EC%8A%A4%ED%8A%B8_batch_first.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip3 install nltk

!pip3 install -U torchtext 

In [ ]:
import nltk
import tensorflow as tf
import torch

In [ ]:
import urllib.request  # not define 에서시는 "from urllib import request"
import pandas as pd
# import urllib
# from urllib import request
#import pandas as pd

In [ ]:
urllib.request.urlretrieve("https://raw.githubusercontent.com/LawrenceDuan/IMDb-Review-Analysis/master/IMDb_Reviews.csv", filename="IMDb_Reviews.csv")

('IMDb_Reviews.csv', <http.client.HTTPMessage at 0x7ff420741b90>)

In [ ]:
df = pd.read_csv('IMDb_Reviews.csv', encoding='latin1')


In [ ]:
# 전체 샘플의 개수를 보겠습니다.
print('전체 샘플의 개수 : {}'.format(len(df)))

전체 샘플의 개수 : 50000


In [ ]:
# 전체 샘플의 개수는 50,000개입니다. 25,000개씩 분리하여 훈련 데이터와 테스트 데이터로 분리해보겠습니다.

train_df = df[:25000]
test_df = df[25000:]

In [ ]:
# 상위 25,000개의 행은 train_df에 하위 25,000개의 행은 test_df에 저장하였습니다.
train_df.to_csv("train_data.csv", index=False)
test_df.to_csv("test_data.csv", index=False)

In [ ]:
from torchtext.legacy import data # torchtext.data 임포트

# 필드 정의
TEXT = data.Field(sequential=True,
                  use_vocab=True,
                  tokenize=str.split,
                  lower=True,
                  batch_first=True, # <== 이 부분을 True로 합니다.
                  fix_length=20)
 # batch_first : 미니 배치 차원을 맨 앞으로 하여 데이터를 불러올 것인지 여부. (False가 기본값)
LABEL = data.Field(sequential=False,
                   use_vocab=False,
                   batch_first=False,
                   is_target=True)

 현 위치의 위,아래 코드 중에서 하나만 실행!!!

 필드 정의할 때 batch_first=True 인자를 넣느냐, 넣지 않느냐의 차이!!!

In [ ]:
#
# 필드 재정의하기(torchtext.data)
####################################

from torchtext.legacy import data # torchtext.data 임포트
# 필드 정의
#TEXT = data.Field(sequential=True,
#                  use_vocab=True,
#                  tokenize=str.split,
#                  lower=True,
#                  " batch_first=True, " # <== 이 부분을 True로 합니다.
#                  fix_length=20)
# batch_first : 미니 배치 차원을 맨 앞으로 하여 데이터를 불러올 것인지 여부. (False가 기본값)
#LABEL = data.Field(sequential=False,
#                   use_vocab=False,
#                   batch_first=False,
#                   is_target=True)
# 필드 정의
TEXT = data.Field(sequential=True,
                  use_vocab=True,
                  tokenize=str.split,
                  lower=True,      # TEXT 필드에서 batch_first = True 인자를 제거하였습니다. 
                  fix_length=20)   # 기본값이 False이므로 batch_first의 값이 False가 됩니다..

LABEL = data.Field(sequential=False,
                   use_vocab=False,
                   batch_first=False,
                   is_target=True)

 -- 3. 데이터셋 / 단어 집합 / 데이터로더 만들기 --

In [ ]:
from torchtext.legacy.data import TabularDataset
from torchtext.legacy.data import Iterator

# TabularDataset은 데이터를 불러오면서 필드에서 정의했던 토큰화 방법으로 토큰화를 수행합니다.
train_data, test_data = TabularDataset.splits(
        path='.', train='train_data.csv', test='test_data.csv', format='csv',
        fields=[('text', TEXT), ('label', LABEL)], skip_header=True)

# 정의한 필드에 .build_vocab() 도구를 사용하면 단어 집합을 생성합니다.
TEXT.build_vocab(train_data, min_freq=10, max_size=10000) # 10,000개의 단어를 가진 단어 집합 생성

# 배치 크기를 정하고 첫번째 배치를 출력해보겠습니다.
batch_size = 5
train_loader = Iterator(dataset=train_data, batch_size = batch_size)
batch = next(iter(train_loader)) # 첫번째 미니배치

In [ ]:
print(batch.text)

tensor([[   2, 1818,   10,   11, 3126],
        [ 308,   10,    7,    7,    2],
        [  12,   20,  314,   87,  348],
        [ 259,  513,   30,    6, 2461],
        [  91,   37,    5,   66,   35],
        [   5,    9,    2,    3, 4697],
        [   0,  350,   78,  163, 1937],
        [ 129,   18,  553,    0,  144],
        [   3,   24,  125,    5,  353],
        [   0, 1369,    5, 5760,   10],
        [   0,  312,   36, 1865,  422],
        [ 262,    6,  289, 7382,  138],
        [   0,  109,   43,    8,   48],
        [3815,   11, 2938,   10,    3],
        [  20,   15,    0, 1243,  739],
        [   7,   81,  945,    9,  158],
        [   0,  258, 3911,  235,   27],
        [   2,    3,    4,   10,  205],
        [ 417,  109, 3905, 5859,    0],
        [ 974,  440,    0,    3,   13]])


In [ ]:
print(batch.text.shape)

torch.Size([20, 5])
